## Part 2

In [4]:
import pandas as pd
import os
from keplergl import KeplerGl
from pyproj import CRS
import numpy as np
from matplotlib import pyplot as plt

In [6]:
df = pd.read_csv('new_york_data.csv', index_col = 0)

C:\Users\aaron\AppData\Local\Temp\ipykernel_9840\2076588821.py:1: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('new_york_data.csv', index_col = 0)


In [8]:
df.drop(columns = {'_merge'}, inplace = True)

In [10]:
df = df.sort_values(by = 'date')

In [12]:
df.head()

ride_id  rideable_type               started_at  \
337491  FB33E3D8F21E2941  electric_bike  2022-01-01 01:49:37.374   
934741  755337295F178067  electric_bike  2022-01-01 03:21:09.754   
227204  C62CA87E3A475ADD   classic_bike  2022-01-01 08:38:18.156   
913722  CD7A2098AFCD5514   classic_bike  2022-01-01 10:33:58.529   
127371  3F9E0C51F49F78A3  electric_bike  2022-01-01 20:05:19.592   

                       ended_at      start_station_name start_station_id  \
337491  2022-01-01 01:57:50.346   Canal St & Rutgers St          5303.08   
934741  2022-01-01 03:49:33.047  Lewis Ave & Madison St          4425.02   
227204  2022-01-01 08:47:54.213  Carlton Ave & Park Ave          4732.04   
913722  2022-01-01 10:47:05.197     W 100 St & Broadway          7580.01   
127371  2022-01-01 20:12:00.661         W 18 St & 6 Ave          6064.08   

               end_station_name end_station_id  start_lat  start_lng  \
337491   Norfolk St & Broome St        5374.01  40.714275 -73.989900   
934741  Columbia St & Degraw St        4422.04  40.686312 -73.935775   
227204  Emerson Pl & Myrtle Ave        4683.02  40.695807 -73.973556   
913722       W 67 St & Broadway        7116.04  40.797372 -73.970412   
127371          W 42 St & 6 Ave        6517.08  40.739713 -73.994564   

          end_lat    end_lng member_casual        date  avgTemp  
337491  40.717227 -73.988021        casual  2022-01-01     11.6  
934741  40.685930 -74.002424        member  2022-01-01     11.6  
227204  40.693631 -73.962236        casual  2022-01-01     11.6  
913722  40.774925 -73.982666        casual  2022-01-01     11.6  
127371  40.754920 -73.984550        member  2022-01-01     11.6

In [14]:
df.shape

(29838166, 15)

## Part 3

In [18]:
# Create a value column and group by start and end station 

df['value'] = 1
df_group = df.groupby(['start_station_name', 'end_station_name'])['value'].count().reset_index()

In [20]:
df_group.rename(columns = {'value': 'trips'}, inplace = True)

In [22]:
df_group.head()

start_station_name  end_station_name  trips
0   1 Ave & E 110 St  1 Ave & E 110 St    791
1   1 Ave & E 110 St   1 Ave & E 18 St      2
2   1 Ave & E 110 St   1 Ave & E 30 St      4
3   1 Ave & E 110 St   1 Ave & E 39 St      1
4   1 Ave & E 110 St   1 Ave & E 44 St     12

In [24]:
df_group.shape

(1013397, 3)

In [26]:
# Creating and merging subsets into the dataframe above

df_sub1 = df[['start_station_name', 'start_lat', 'start_lng']]

In [30]:
df_sub1 = df_sub1.drop_duplicates()

In [32]:
df_sub1.head()

start_station_name  start_lat  start_lng
337491   Canal St & Rutgers St  40.714275 -73.989900
934741  Lewis Ave & Madison St  40.686312 -73.935775
227204  Carlton Ave & Park Ave  40.695807 -73.973556
913722     W 100 St & Broadway  40.797372 -73.970412
127371         W 18 St & 6 Ave  40.739713 -73.994564

In [34]:
df_sub1.shape

(4067371, 3)

In [36]:
print(df_sub1['start_station_name'].nunique())

1761


In [40]:
# Adding new columns to standardize latitude and longitude

df_sub1['start_lat+'] = df_sub1.groupby(['start_station_name'])['start_lat'].transform(np.mean)

C:\Users\aaron\AppData\Local\Temp\ipykernel_9840\1577166103.py:3: FutureWarning: The provided callable <function mean at 0x00000295EE48CD60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_sub1['start_lat+'] = df_sub1.groupby(['start_station_name'])['start_lat'].transform(np.mean)


In [42]:
df_sub1['start_lon'] = df_sub1.groupby(['start_station_name'])['start_lng'].transform(np.mean)

C:\Users\aaron\AppData\Local\Temp\ipykernel_9840\2927703104.py:1: FutureWarning: The provided callable <function mean at 0x00000295EE48CD60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_sub1['start_lon'] = df_sub1.groupby(['start_station_name'])['start_lng'].transform(np.mean)


In [44]:
# Removing redundant columns

df_sub1.drop(columns = {'start_lat', 'start_lng'}, inplace = True)

df_sub1.rename(columns = {'start_lat+': 'start_lat'}, inplace = True)

In [50]:
df_sub1 = df_sub1.drop_duplicates()

In [52]:
df_sub1.head()

start_station_name  start_lat  start_lon
337491   Canal St & Rutgers St  40.714307 -73.989912
934741  Lewis Ave & Madison St  40.686232 -73.935735
227204  Carlton Ave & Park Ave  40.695720 -73.973530
913722     W 100 St & Broadway  40.797374 -73.970347
127371         W 18 St & 6 Ave  40.739635 -73.994569

In [54]:
df_sub1.shape

(1762, 3)

In [56]:
# Repeating previous steps for the second subset

df_sub2 = df[['end_station_name', 'end_lat', 'end_lng']]

In [58]:
df_sub2 = df_sub2.drop_duplicates()

In [60]:
df_sub2.head()

end_station_name    end_lat    end_lng
337491   Norfolk St & Broome St  40.717227 -73.988021
934741  Columbia St & Degraw St  40.685930 -74.002424
227204  Emerson Pl & Myrtle Ave  40.693631 -73.962236
913722       W 67 St & Broadway  40.774925 -73.982666
127371          W 42 St & 6 Ave  40.754920 -73.984550

In [62]:
df_sub2.shape

(3205, 3)

In [64]:
print(df_sub2['end_station_name'].nunique())

1841


In [66]:
# Adding new columns to standardize latitude and longitude

df_sub2['end_lat+'] = df_sub2.groupby(['end_station_name'])['end_lat'].transform(np.mean)

C:\Users\aaron\AppData\Local\Temp\ipykernel_9840\3489016560.py:3: FutureWarning: The provided callable <function mean at 0x00000295EE48CD60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_sub2['end_lat+'] = df_sub2.groupby(['end_station_name'])['end_lat'].transform(np.mean)


In [68]:
df_sub2['end_lon'] = df_sub2.groupby(['end_station_name'])['end_lng'].transform(np.mean)

C:\Users\aaron\AppData\Local\Temp\ipykernel_9840\766103063.py:1: FutureWarning: The provided callable <function mean at 0x00000295EE48CD60> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_sub2['end_lon'] = df_sub2.groupby(['end_station_name'])['end_lng'].transform(np.mean)


In [70]:
# Removing redundant columns

df_sub2.drop(columns = {'end_lat', 'end_lng'}, inplace = True)

df_sub2.rename(columns = {'end_lat+': 'end_lat'}, inplace = True)

In [72]:
df_sub2 = df_sub2.drop_duplicates()

In [74]:
df_sub2.head()

end_station_name    end_lat    end_lon
337491   Norfolk St & Broome St  40.717227 -73.988021
934741  Columbia St & Degraw St  40.685930 -74.002424
227204  Emerson Pl & Myrtle Ave  40.693631 -73.962236
913722       W 67 St & Broadway  40.774925 -73.982666
127371          W 42 St & 6 Ave  40.754920 -73.984550

In [76]:
df_sub2.shape

(1842, 3)

In [79]:
# Merging dataframes

df_m = df_group.merge(df_sub1, how = 'outer', on = "start_station_name", indicator = True)

In [82]:
df_m['_merge'].value_counts(dropna = False)

_merge
both          1013397
right_only          1
left_only           0
Name: count, dtype: int64

In [85]:
df_m = df_m[df_m['_merge'] =='both']

In [88]:
df_m.drop(columns = {'_merge'}, inplace = True)

In [91]:
df_f = df_m.merge(df_sub2, how = 'outer', on = "end_station_name", indicator = True)

In [93]:
df_f['_merge'].value_counts(dropna = False)

_merge
both          1013397
right_only          1
left_only           0
Name: count, dtype: int64

In [95]:
df_f = df_f[df_f['_merge'] =='both']

In [97]:
df_f.drop(columns = {'_merge'}, inplace = True)

In [99]:
df_f.head()

start_station_name  end_station_name  trips  start_lat  start_lon  \
0   1 Ave & E 110 St  1 Ave & E 110 St  791.0  40.792362 -73.938196   
1    1 Ave & E 16 St  1 Ave & E 110 St    5.0  40.732255 -73.981639   
2    1 Ave & E 18 St  1 Ave & E 110 St    6.0  40.734081 -73.980427   
3    1 Ave & E 30 St  1 Ave & E 110 St   25.0  40.741519 -73.975316   
4    1 Ave & E 39 St  1 Ave & E 110 St   78.0  40.747330 -73.971051   

     end_lat  end_lon  
0  40.792327 -73.9383  
1  40.792327 -73.9383  
2  40.792327 -73.9383  
3  40.792327 -73.9383  
4  40.792327 -73.9383

In [101]:
df_f.shape

(1013397, 7)

## Part 4-6

In [106]:
# Create KeplerGl instance

m = KeplerGl(height = 700, data={"data_1": df_f})

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [108]:
m

KeplerGl(data={'data_1':                   start_station_name       end_station_name   trips  \
0             …

The changes that I made to this map incloud adding an ark inbetween the start and end points, as well as changing the colors of both the points and the arks. I did this so that the map is easies to understand. The color of the arks change from purple to yellow to show the start and end of each trip. And the points also shares this purple and yellow color scheme, though the points have a darker color to help make them stand out more.

After filtering the map to only show trips that were taken more than 3,000 times during 2022, this is what I found. A lot of trips are located arounf Central Park, a massive location that tones of people visit every day. Yet there are even more parks dotted around the west side of Manhattan jutting out into the river. The South West corner of Manhattan has lots of popular torist locations, like the World Trade Center. And Other high trip routs are located near schools, hospitals, and libraries.

## Part 7

In [116]:
# Saving the map

config = m.config

In [119]:
import json
with open("config.json", "w") as outfile:
    json.dump(config, outfile)

In [122]:
m.save_to_html(file_name = 'New York Bike Trips Aggregated.html', read_only = False, config = config)

Map saved to New York Bike Trips Aggregated.html!
